In [40]:
api_key = '8deafecd3404b23ad954c15380b7e3e32e00efad'
import pandas as pd
import requests
import time

## Decennial Census

Name of the variables: https://api.census.gov/data/2000/dec/sf1/variables.html

In [ ]:
merged_df = pd.DataFrame()

for year in [2000,2010]:

    endpoint = f'http://api.census.gov/data/{year}/dec/sf1?get=P001001,H004001,H004002,H004003,NAME&for=zip%20code%20tabulation%20area:*&key={api_key}'

    response = requests.get(endpoint)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
    else:
        print(f'Request failed with status code: {response.status_code}')

    df = pd.DataFrame(data[1:], columns=data[0])
    df['Year'] = year
    df['ZTCA5'] = df['zip code tabulation area']
    columns_to_convert = ['H004001', 'H004002', 'H004003']

    for column in columns_to_convert:
        df[column] = pd.to_numeric(df[column], errors='coerce')
    df['pct_owned'] = df['H004002'] / df['H004001'] *100
    df['pct_rent'] = df['H004003'] / df['H004001'] *100

    df = df[['Year', 'ZTCA5','NAME', 'pct_owned', 'pct_rent']]

    merged_df = pd.concat([merged_df,df],axis=0)

In [ ]:
merged_df

,Year,ZTCA5,NAME,pct_owned,pct_rent
0,2000,00601,ZCTA5 00601,71.891433,28.108567
1,2000,00602,ZCTA5 00602,80.628698,19.371302
2,2000,00603,ZCTA5 00603,65.747703,34.252297
3,2000,00604,ZCTA5 00604,65.923283,34.076717
4,2000,00606,ZCTA5 00606,72.776950,27.223050
...,...,...,...,...,...
33115,2010,01841,ZCTA5 01841,25.434616,4.607973
33116,2010,01851,ZCTA5 01851,35.517654,9.285857
33117,2010,01852,ZCTA5 01852,35.902381,10.993250
33118,2010,01886,ZCTA5 01886,70.818885,19.658576


In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/economic/'
merged_df.to_csv(path+'owned_rent_ztca5_dc.csv', index=False)

## Population Density:

Calculated using the population from the Decennial Census and land area in [Census Dataset](https://covid19.census.gov/datasets/21843f238cbb46b08615fc53e19e0daf/explore?showTable=true)

P001001: Total population

P003003: Total!!Population of one race!!White alone

P003004: Total!!Population of one race!!Black or African American alone

P003005: Total!!Population of one race!!American Indian and Alaska Native alone

P003006: Total!!Population of one race!!Asian alone

P004002: Total!!Hispanic or Latino

In [ ]:
merged_df = pd.DataFrame()

variables = 'P001001,P003003,P003004,P003005,P003006,P004002'

column_names = [
    'Total population',
    'White',
    'Black or African American',
    'American Indian and Alaska Native',
    'Asian',
    'Hispanic or Latino',
    'Name',
    'ZTCA5'
]

for year in [2000,2010]:

    endpoint = f'http://api.census.gov/data/{year}/dec/sf1?get={variables},NAME&for=zip%20code%20tabulation%20area:*&key={api_key}'

    response = requests.get(endpoint)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        print(data)
    else:
        print(f'Request failed with status code: {response.status_code}')

    df = pd.DataFrame(data[1:], columns=data[0])
    df = df.rename(columns=dict(zip(df.columns, column_names)))

    df['Year'] = year

    columns_to_convert = ['Total population','White','Black or African American','American Indian and Alaska Native','Asian','Hispanic or Latino']

    for column in columns_to_convert:
        df[column] = pd.to_numeric(df[column], errors='coerce')

    df['pct_white'] = df['White'] / df['Total population'] * 100
    df['pct_black'] = df['Black or African American'] / df['Total population'] * 100
    df['pct_indian'] = df['American Indian and Alaska Native'] / df['Total population'] * 100
    df['pct_asian'] = df['Asian'] / df['Total population'] * 100
    df['pct_hispanic'] = df['Hispanic or Latino'] / df['Total population'] * 100

    df = df[['Year', 'ZTCA5','Name', 'Total population','pct_white', 'pct_black','pct_indian','pct_asian','pct_hispanic']]

    merged_df = pd.concat([merged_df,df],axis=0)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
race_demographics_dc = merged_df[['Year','ZTCA5','Name','pct_white','pct_black','pct_indian','pct_asian','pct_hispanic']]
master_path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
race_demographics_dc.to_csv(master_path+'race_demographics_ztca5_dc.csv',index=False)

Merge with a csv with areas of ZIP code

In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/population/density/'
df = pd.read_excel(path + 'tigerweb_tab20_zcta5_2010_us.xlsx')

In [ ]:
df = df[['ZCTA5','NAME','POP100','AREALAND']]

In [ ]:
columns_to_convert = ['POP100','AREALAND']

for column in columns_to_convert:
    df[column] = pd.to_numeric(df[column], errors='coerce')

# Define the conversion factor
conversion_factor = 0.00000038610215855

# Perform the conversion by dividing the values by the conversion factor
df['AREALAND_MILES'] = df['AREALAND'] * conversion_factor


In [ ]:
df['DENSITY'] = df['POP100'] / df['AREALAND_MILES']
df['YEAR'] = 2010

In [ ]:
master_path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
df.to_csv(master_path+'pop_density_zcta5_dc.csv',index=False)

In [ ]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/raw data/population/density/'
land_area = pd.read_csv(path+'Average_Household_Size_and_Population_Density_-_County.csv')

In [ ]:
land_area['GEOID'] = land_area['GEOID'].astype(str).str.zfill(5)
land_area['ALAND'] = land_area['ALAND'] / 1000

,OBJECTID,COUNTYNS,GEOID,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,...,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
0,1,161526,01001,1539602.123,25706961,Autauga County,Alabama,2.59,0.05,2.59,...,55200,NaN,35.853419,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.066037,0.150256
1,2,161527,01003,4117546.676,1133055836,Baldwin County,Alabama,2.61,0.04,2.66,...,208107,NaN,50.541504,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,4.483746,0.409904
2,3,161528,01005,2292144.655,50538698,Barbour County,Alabama,2.49,0.07,2.44,...,25782,NaN,11.247981,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.695262,0.223270
3,4,161529,01007,1612167.481,9602089,Bibb County,Alabama,2.99,0.14,3.05,...,22527,NaN,13.973114,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,1.887514,0.156473
4,5,161530,01009,1670103.911,15015423,Blount County,Alabama,2.77,0.05,2.85,...,57645,NaN,34.515816,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.423552,0.164405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,3216,1804553,72145,118777.649,57795019,Vega Baja Municipio,Puerto Rico,2.93,0.09,2.93,...,53371,NaN,449.335380,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.485650,0.010381
3216,3217,1804554,72147,131541.395,552192819,Vieques Municipio,Puerto Rico,3.55,0.38,3.03,...,8771,NaN,66.678630,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.675819,0.012181
3217,3218,1804555,72149,92298.569,3622639,Villalba Municipio,Puerto Rico,2.96,0.10,3.09,...,22993,NaN,249.115455,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.431598,0.008189
3218,3219,1804556,72151,143005.179,72592521,Yabucoa Municipio,Puerto Rico,2.91,0.10,2.90,...,34149,NaN,238.795547,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.676628,0.012229


In [ ]:
land_area = land_area.rename(columns={'GEOID': 'FIPS'})
land_area

,OBJECTID,COUNTYNS,FIPS,ALAND,AWATER,NAME,State,B25010_001E,B25010_001M,B25010_002E,...,B01001_001E,B01001_001M,B01001_calc_PopDensity,created_user,created_date,last_edited_user,last_edited_date,B01001_calc_PopDensityM,SHAPE_Length,SHAPE_Area
0,1,161526,01001,1539602.123,25706961,Autauga County,Alabama,2.59,0.05,2.59,...,55200,NaN,35.853419,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.066037,0.150256
1,2,161527,01003,4117546.676,1133055836,Baldwin County,Alabama,2.61,0.04,2.66,...,208107,NaN,50.541504,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,4.483746,0.409904
2,3,161528,01005,2292144.655,50538698,Barbour County,Alabama,2.49,0.07,2.44,...,25782,NaN,11.247981,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.695262,0.223270
3,4,161529,01007,1612167.481,9602089,Bibb County,Alabama,2.99,0.14,3.05,...,22527,NaN,13.973114,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,1.887514,0.156473
4,5,161530,01009,1670103.911,15015423,Blount County,Alabama,2.77,0.05,2.85,...,57645,NaN,34.515816,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,2.423552,0.164405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,3216,1804553,72145,118777.649,57795019,Vega Baja Municipio,Puerto Rico,2.93,0.09,2.93,...,53371,NaN,449.335380,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.485650,0.010381
3216,3217,1804554,72147,131541.395,552192819,Vieques Municipio,Puerto Rico,3.55,0.38,3.03,...,8771,NaN,66.678630,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.675819,0.012181
3217,3218,1804555,72149,92298.569,3622639,Villalba Municipio,Puerto Rico,2.96,0.10,3.09,...,22993,NaN,249.115455,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.431598,0.008189
3218,3219,1804556,72151,143005.179,72592521,Yabucoa Municipio,Puerto Rico,2.91,0.10,2.90,...,34149,NaN,238.795547,esri_demographics,2020/04/01 20:53:36+00,esri_demographics,2020/04/01 20:53:36+00,0.0,0.676628,0.012229


In [ ]:
pop_density = pd.merge(merged_df[['Year', 'FIPS', 'Name', 'Total population']], land_area[['FIPS', 'ALAND']], on='FIPS', how='inner')
pop_density

,Year,FIPS,Name,Total population,ALAND
0,2000,01001,"Autauga County, Alabama",43671,1.539602e+06
1,2010,01001,"Autauga County, Alabama",54571,1.539602e+06
2,2000,01003,"Baldwin County, Alabama",140415,4.117547e+06
3,2010,01003,"Baldwin County, Alabama",182265,4.117547e+06
4,2000,01005,"Barbour County, Alabama",29038,2.292145e+06
...,...,...,...,...,...
6425,2010,02195,"Petersburg Census Area, Alaska",3815,7.512558e+06
6426,2010,02198,"Prince of Wales-Hyder Census Area, Alaska",5559,1.363052e+07
6427,2010,02275,"Wrangell City and Borough, Alaska",2369,6.619997e+06
6428,2010,02230,"Skagway Municipality, Alaska",968,1.123881e+06


In [ ]:
pop_density.dtypes

Year                  int64
FIPS                 object
Name                 object
Total population      int64
ALAND               float64
dtype: object

In [ ]:
pop_density['pop_density'] = pop_density['Total population'] / pop_density['ALAND']

In [ ]:
pop_density.to_csv(master_path+'pop_density_dc.csv',index=False)

## American Community Survey (2005-2020)

### 1.  Race Demographics By ZIP Code

#### 2011-2021

The list of variables is listed in this
[Google Sheet](https://docs.google.com/spreadsheets/d/1-r8Edga5FG1xg4fL4rTmi6UOf6-wLVokgu7N3-QaAZ4/edit?usp=sharing)


In [41]:
def generate_df(years, variables, column_names):
  merged_df = pd.DataFrame()

  for year in years:
      endpoint = f'https://api.census.gov/data/{year}/acs/acs5/profile?get=NAME,{variables}&for=zip%20code%20tabulation%20area:*&key={api_key}'

      response = requests.get(endpoint)
      # Check if the request was successful
      if response.status_code == 200:
          data = response.json()  # Parse the response as JSON
          print(year)
          print(data)  # Display the retrieved data
      else:
          print(year)
          print(f'Request failed with status code: {response.status_code}')

      # Rename the columns and add the year
      df = pd.DataFrame(data[1:], columns=data[0])
      df = df.rename(columns=dict(zip(df.columns, column_names)))
      df['Year'] = year

      # Convert the data to numeric
      columns_to_convert = column_names[1:-2]
      for column in columns_to_convert:
          df[column] = pd.to_numeric(df[column], errors='coerce')

      # If the year is 2005 or 2006, calculate the percentage from the estimate
      if year == 2005 or year == 2006:
          df['pct_white'] = (df['White'] / df['Total population'] * 100).round(2)
          df['pct_black'] = (df['Black or African American'] / df['Total population'] * 100).round(2)
          df['pct_indian'] = (df['American Indian and Alaska Native'] / df['Total population'] * 100).round(2)
          df['pct_asian'] = (df['Asian'] / df['Total population'] * 100).round(2)
          df['pct_hispanic'] = (df['Hispanic or Latino'] / df['Total population'] * 100).round(2)

      merged_df = pd.concat([merged_df,df],axis=0)

      time.sleep(5)

  return merged_df

In [51]:
years1 = range(2011,2017)
variables1 = 'DP05_0059PE,DP05_0060PE,DP05_0061PE,DP05_0062PE,DP05_0066PE'

column_names = ['Name','pct_white','pct_black','pct_indian','pct_asian','pct_hispanic','State','ZIP']

years2 = range(2017,2020)
variables2 = 'DP05_0064PE,DP05_0065PE,DP05_0066PE,DP05_0067PE,DP05_0071PE'

years3 = [2020,2021]
column_names2 = ['Name','pct_white','pct_black','pct_indian','pct_asian','pct_hispanic','ZIP']

df1 = generate_df(years1, variables1, column_names)
df2 = generate_df(years2, variables2, column_names)
df3 = generate_df(years3, variables2, column_names2)

Output hidden; open in https://colab.research.google.com to view.

In [47]:
# Concatenate dataframes
race_demographics_acs = pd.concat([df1, df2, df3])

# Reset the index of the concatenated dataframe
race_demographics_acs.reset_index(drop=True, inplace=True)

race_demographics_acs

,Name,pct_white,pct_black,pct_indian,pct_asian,pct_hispanic,State,ZIP,Year
0,ZCTA5 17774,99.1,0.0,3.4,0.0,0.3,42,17774,2011
1,ZCTA5 17815,94.8,3.6,0.6,1.1,2.1,42,17815,2011
2,ZCTA5 17827,99.3,0.0,0.0,0.7,2.5,42,17827,2011
3,ZCTA5 17837,87.5,7.2,0.8,3.0,5.1,42,17837,2011
4,ZCTA5 17846,97.9,1.3,0.0,0.3,1.5,42,17846,2011
...,...,...,...,...,...,...,...,...,...
364969,ZCTA5 99923,100.0,0.0,0.0,0.0,0.0,NaN,99923,2021
364970,ZCTA5 99925,61.7,0.4,53.1,5.5,4.1,NaN,99925,2021
364971,ZCTA5 99926,7.1,0.2,91.8,1.5,3.3,NaN,99926,2021
364972,ZCTA5 99927,100.0,0.0,0.0,0.0,0.0,NaN,99927,2021


#### 2005-2010

Use the zcta5 to county file for 2005-2010 duration

In [98]:
year_dict = {
    2005: ['DP01_0059E', 'DP01_0060E', 'DP01_0061E', 'DP01_0062E', 'DP01_0066E', 'DP01_0058E', 'DP03_0094E', 'DP03_0061E', 'DP04_0041E', 'DP04_0042E', 'DP04_0040E'],
    2006: ['DP05_0059E', 'DP05_0060E', 'DP05_0061E', 'DP05_0062E', 'DP05_0066E', 'DP05_0058E', 'DP03_0094E', 'DP03_0061E', 'DP04_0041E', 'DP04_0042E', 'DP04_0040E'],
    2007: ['DP05_0059PE', 'DP05_0060PE', 'DP05_0061PE', 'DP05_0062PE', 'DP05_0066PE', 'DP03_0096PE', 'DP03_0063E', 'DP04_0045PE', 'DP04_0046PE'],
    2008: ['DP05_0059PE', 'DP05_0060PE', 'DP05_0061PE', 'DP05_0062PE', 'DP05_0066PE', 'DP03_0096PE', 'DP03_0063E', 'DP04_0045PE', 'DP04_0046PE'],
    2009: ['DP05_0059PE', 'DP05_0060PE', 'DP05_0061PE', 'DP05_0062PE', 'DP05_0066PE', 'DP03_0103PE', 'DP03_0063E', 'DP04_0045PE', 'DP04_0046PE'],
    2010: ['DP05_0059PE', 'DP05_0060PE', 'DP05_0061PE', 'DP05_0062PE', 'DP05_0066PE', 'DP03_0119PE', 'DP03_0062E', 'DP04_0045PE', 'DP04_0046PE']
}

In [99]:
def generate_df_county(years, column_names):
  merged_df = pd.DataFrame()

  for year in years:

      variables = ','.join(year_dict[year])
      endpoint = f'https://api.census.gov/data/{year}/acs/acs1/profile?get=NAME,{variables}&for=county:*&key={api_key}'

      response = requests.get(endpoint)
      # Check if the request was successful
      if response.status_code == 200:
          data = response.json()  # Parse the response as JSON
          print(year)
          print(data)  # Display the retrieved data
      else:
          print(year)
          print(f'Request failed with status code: {response.status_code}')

      # Rename the columns and add the year
      df = pd.DataFrame(data[1:], columns=data[0])
      df = df.rename(columns=dict(zip(df.columns, column_names)))
      df['Year'] = year
      df['State']  = df['State'].astype(str).str.zfill(2)
      df['County'] = df['County'].astype(str).str.zfill(3)
      df['FIPS'] = df['State'] + df['County']

      # Convert the data to numeric
      columns_to_convert = column_names[1:-2]
      for column in columns_to_convert:
          df[column] = pd.to_numeric(df[column], errors='coerce')

      # If the year is 2005 or 2006, calculate the percentage from the estimate
      if year == 2005 or year == 2006:
          df['pct_white'] = (df['White'] / df['Total population'] * 100).round(2)
          df['pct_black'] = (df['Black'] / df['Total population'] * 100).round(2)
          df['pct_indian'] = (df['Indian'] / df['Total population'] * 100).round(2)
          df['pct_asian'] = (df['Asian'] / df['Total population'] * 100).round(2)
          df['pct_hispanic'] = (df['Hispanic'] / df['Total population'] * 100).round(2)

          df['pct_owned'] = (df['Owned'] / df['Total Housing Units'] * 100).round(2)
          df['pct_rent'] = (df['Rent'] / df['Total Housing Units'] * 100).round(2)

      merged_df = pd.concat([merged_df,df],axis=0)

      time.sleep(5)

  return merged_df

In [100]:
years1 = [2005,2006]
variables1 = 'DP05_0059PE,DP05_0060PE,DP05_0061PE,DP05_0062PE,DP05_0066PE'
column_names1 = ['Name','White','Black','Indian','Asian','Hispanic','Total population','Poverty','Median Household Income','Owned','Rent','Total Housing Units','State','County']

years2 = [2007,2008,2009,2010]
variables2 = 'DP05_0064PE,DP05_0065PE,DP05_0066PE,DP05_0067PE,DP05_0071PE'
column_names2 = ['Name','pct_white','pct_black','pct_indian','pct_asian','pct_hispanic','Poverty','Median Household Income','pct_owned','pct_rent','State','County']

df1 = generate_df_county(years1, column_names1)
df2 = generate_df_county(years2, column_names2)

2005
[['NAME', 'DP01_0059E', 'DP01_0060E', 'DP01_0061E', 'DP01_0062E', 'DP01_0066E', 'DP01_0058E', 'DP03_0094E', 'DP03_0061E', 'DP04_0041E', 'DP04_0042E', 'DP04_0040E', 'state', 'county'], ['Baldwin County, Alabama', '141862', '16304', '1512', '442', '3633', '160354', '10.2', '42119', '50877', '15203', '66080', '01', '003'], ['Calhoun County, Alabama', '86602', '21628', '1075', '1050', '2222', '109762', '12.3', '36243', '33968', '13018', '46986', '01', '015'], ['Cullman County, Alabama', '77056', '1115', '741', '-999999999', '-999999999', '78888', '-999999999.0', '37153', '21206', '8713', '29919', '01', '043'], ['DeKalb County, Alabama', '61414', '1268', '1044', '-999999999', '6046', '66407', '18.0', '29053', '18872', '6744', '25616', '01', '049'], ['Elmore County, Alabama', '54852', '11856', '796', '449', '1198', '68057', '-999999999.0', '45802', '19836', '5337', '25173', '01', '051'], ['Etowah County, Alabama', '84730', '14612', '680', '505', '2372', '101151', '12.6', '32507', '31206

In [59]:
df05_10 = pd.concat([df1, df2])
df05_10

,Name,White,Black,Indian,Asian,Hispanic,Total population,Poverty,Median Household Income,Owned,...,County,Year,FIPS,pct_white,pct_black,pct_indian,pct_asian,pct_hispanic,pct_owned,pct_rent
0,"Baldwin County, Alabama",141862.0,16304.0,1512.0,442.0,3633.0,160354.0,10.2,42119,50877.0,...,003,2005,01003,88.47,10.17,9.400000e-01,2.800000e-01,2.27,76.99,23.01
1,"Calhoun County, Alabama",86602.0,21628.0,1075.0,1050.0,2222.0,109762.0,12.3,36243,33968.0,...,015,2005,01015,78.90,19.70,9.800000e-01,9.600000e-01,2.02,72.29,27.71
2,"Cullman County, Alabama",77056.0,1115.0,741.0,-999999999.0,-999999999.0,78888.0,-999999999.0,37153,21206.0,...,043,2005,01043,97.68,1.41,9.400000e-01,-1.267620e+06,-1267619.92,70.88,29.12
3,"DeKalb County, Alabama",61414.0,1268.0,1044.0,-999999999.0,6046.0,66407.0,18.0,29053,18872.0,...,049,2005,01049,92.48,1.91,1.570000e+00,-1.505865e+06,9.10,73.67,26.33
4,"Elmore County, Alabama",54852.0,11856.0,796.0,449.0,1198.0,68057.0,-999999999.0,45802,19836.0,...,051,2005,01051,80.60,17.42,1.170000e+00,6.600000e-01,1.76,78.80,21.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,"Toa Alta Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,37.3,25432,NaN,...,135,2010,72135,85.40,50.80,2.700000e+00,-1.000000e+09,99.80,73.10,26.90
814,"Toa Baja Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,32.6,23296,NaN,...,137,2010,72137,77.40,36.00,-1.000000e+09,-1.000000e+09,98.80,80.40,19.60
815,"Trujillo Alto Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,32.6,29120,NaN,...,139,2010,72139,89.90,42.50,-1.000000e+09,-1.000000e+09,99.60,71.10,28.90
816,"Mayagüez Municipio, Puerto Rico",NaN,NaN,NaN,NaN,NaN,NaN,47.1,15381,NaN,...,097,2010,72097,64.50,18.00,-1.000000e+09,-1.000000e+09,97.60,61.70,38.30


In [60]:
df_county = df05_10[['Name','pct_white','pct_black','pct_indian','pct_asian','pct_hispanic','FIPS','Year']]
df_county

,Name,pct_white,pct_black,pct_indian,pct_asian,pct_hispanic,FIPS,Year
0,"Baldwin County, Alabama",88.47,10.17,9.400000e-01,2.800000e-01,2.27,01003,2005
1,"Calhoun County, Alabama",78.90,19.70,9.800000e-01,9.600000e-01,2.02,01015,2005
2,"Cullman County, Alabama",97.68,1.41,9.400000e-01,-1.267620e+06,-1267619.92,01043,2005
3,"DeKalb County, Alabama",92.48,1.91,1.570000e+00,-1.505865e+06,9.10,01049,2005
4,"Elmore County, Alabama",80.60,17.42,1.170000e+00,6.600000e-01,1.76,01051,2005
...,...,...,...,...,...,...,...,...
813,"Toa Alta Municipio, Puerto Rico",85.40,50.80,2.700000e+00,-1.000000e+09,99.80,72135,2010
814,"Toa Baja Municipio, Puerto Rico",77.40,36.00,-1.000000e+09,-1.000000e+09,98.80,72137,2010
815,"Trujillo Alto Municipio, Puerto Rico",89.90,42.50,-1.000000e+09,-1.000000e+09,99.60,72139,2010
816,"Mayagüez Municipio, Puerto Rico",64.50,18.00,-1.000000e+09,-1.000000e+09,97.60,72097,2010


In [61]:
cpath = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/county/'
crosswalk = pd.read_csv(cpath + 'zcta_to_county_2010.csv')

In [62]:
crosswalk['ZCTA5'] = crosswalk['ZCTA5'].astype(str).str.zfill(5)
crosswalk['FIPS'] = crosswalk['STATE'].astype(str).str.zfill(2) + crosswalk['COUNTY'].astype(str).str.zfill(3)

In [64]:
merged_df = df_county.merge(crosswalk, on='FIPS', how='outer')
merged_df

,Name,pct_white,pct_black,pct_indian,pct_asian,pct_hispanic,FIPS,Year,ZCTA5,STATE,...,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,"Baldwin County, Alabama",88.47,10.17,0.94,0.28,2.27,01003,2005.0,36480,1,...,5.250712e+09,4.117522e+09,1.02,0.7,0.64,0.65,0.01,0.01,0.05,0.07
1,"Baldwin County, Alabama",88.47,10.17,0.94,0.28,2.27,01003,2005.0,36502,1,...,5.250712e+09,4.117522e+09,1.20,1.4,11.00,11.05,0.12,0.09,1.79,2.28
2,"Baldwin County, Alabama",88.47,10.17,0.94,0.28,2.27,01003,2005.0,36507,1,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,10.47,7.58,16.67,21.06
3,"Baldwin County, Alabama",88.47,10.17,0.94,0.28,2.27,01003,2005.0,36511,1,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,0.42,0.34,0.25,0.29
4,"Baldwin County, Alabama",88.47,10.17,0.94,0.28,2.27,01003,2005.0,36526,1,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,15.30,12.03,1.91,2.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145606,NaN,NaN,NaN,NaN,NaN,NaN,02198,NaN,99925,2,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,14.73,13.37,0.91,1.42
145607,NaN,NaN,NaN,NaN,NaN,NaN,02198,NaN,99926,2,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,26.26,18.18,2.80,3.39
145608,NaN,NaN,NaN,NaN,NaN,NaN,02198,NaN,99927,2,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,1.69,3.38,3.05,5.80
145609,NaN,NaN,NaN,NaN,NaN,NaN,02275,NaN,99903,2,...,8.966687e+09,6.582411e+09,100.00,100.0,100.00,100.00,1.31,4.83,5.09,6.10


In [ ]:
merged_df['Name'] = 'ZCTA5 '+ merged_df['ZCTA5']
merged_df = merged_df.rename(columns={'ZCTA5': 'ZIPS'})

In [79]:
race_demographics_acs_05 = merged_df[['Name','pct_white','pct_black','pct_indian','pct_asian','pct_hispanic','ZIPS','Year']]

In [80]:
race_demographics_acs_05 = race_demographics_acs_05.dropna()

In [81]:
race_demographics_acs_05['Year'] = race_demographics_acs_05['Year'].astype(int)

In [92]:
race_demographics_acs = race_demographics_acs.drop('State',axis=1)

In [93]:
final_df = pd.concat([race_demographics_acs_05,race_demographics_acs])
final_df

,Name,pct_white,pct_black,pct_indian,pct_asian,pct_hispanic,ZIPS,Year
0,ZCTA5 36480,88.47,10.17,0.94,0.28,2.27,36480,2005
1,ZCTA5 36502,88.47,10.17,0.94,0.28,2.27,36502,2005
2,ZCTA5 36507,88.47,10.17,0.94,0.28,2.27,36507,2005
3,ZCTA5 36511,88.47,10.17,0.94,0.28,2.27,36511,2005
4,ZCTA5 36526,88.47,10.17,0.94,0.28,2.27,36526,2005
...,...,...,...,...,...,...,...,...
364969,ZCTA5 99923,100.00,0.00,0.00,0.00,0.0,99923,2021
364970,ZCTA5 99925,61.70,0.40,53.10,5.50,4.1,99925,2021
364971,ZCTA5 99926,7.10,0.20,91.80,1.50,3.3,99926,2021
364972,ZCTA5 99927,100.00,0.00,0.00,0.00,0.0,99927,2021


In [94]:
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
final_df.to_csv(path + 'race_demographics_zcta5_acs.csv',index=False)

### 2.  Poverty, Median Household Income, Percentage Owned and Rent By ZIP Code

##### 2011-2021

In [96]:
years1 = range(2011,2020)
variables1 = 'DP03_0119PE,DP03_0062E,DP04_0046PE,DP04_0047PE'
column_names1 = ['Name','Poverty','Median Household Income','pct_owned','pct_rent','State','ZIP']

years2 = [2020,2021]
column_names2 = ['Name','Poverty','Median Household Income','pct_owned','pct_rent','ZIP']

df1 = generate_df(years1, variables1, column_names1)
df2 = generate_df(years2, variables1, column_names2)

Output hidden; open in https://colab.research.google.com to view.

In [97]:
df11 = pd.concat([df1,df2])

##### 2005-2010

In [101]:
df05 =  df05_10[['Name','Poverty','Median Household Income','pct_owned','pct_rent','FIPS','Year']]
df05

,Name,Poverty,Median Household Income,pct_owned,pct_rent,FIPS,Year
0,"Baldwin County, Alabama",10.2,42119,76.99,23.01,01003,2005
1,"Calhoun County, Alabama",12.3,36243,72.29,27.71,01015,2005
2,"Cullman County, Alabama",-999999999.0,37153,70.88,29.12,01043,2005
3,"DeKalb County, Alabama",18.0,29053,73.67,26.33,01049,2005
4,"Elmore County, Alabama",-999999999.0,45802,78.80,21.20,01051,2005
...,...,...,...,...,...,...,...
813,"Toa Alta Municipio, Puerto Rico",37.3,25432,73.10,26.90,72135,2010
814,"Toa Baja Municipio, Puerto Rico",32.6,23296,80.40,19.60,72137,2010
815,"Trujillo Alto Municipio, Puerto Rico",32.6,29120,71.10,28.90,72139,2010
816,"Mayagüez Municipio, Puerto Rico",47.1,15381,61.70,38.30,72097,2010


In [102]:
merged_df = df05.merge(crosswalk, on='FIPS', how='outer')
merged_df

,Name,Poverty,Median Household Income,pct_owned,pct_rent,FIPS,Year,ZCTA5,STATE,COUNTY,...,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,"Baldwin County, Alabama",10.2,42119.0,76.99,23.01,01003,2005.0,36480,1,3,...,5.250712e+09,4.117522e+09,1.02,0.7,0.64,0.65,0.01,0.01,0.05,0.07
1,"Baldwin County, Alabama",10.2,42119.0,76.99,23.01,01003,2005.0,36502,1,3,...,5.250712e+09,4.117522e+09,1.20,1.4,11.00,11.05,0.12,0.09,1.79,2.28
2,"Baldwin County, Alabama",10.2,42119.0,76.99,23.01,01003,2005.0,36507,1,3,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,10.47,7.58,16.67,21.06
3,"Baldwin County, Alabama",10.2,42119.0,76.99,23.01,01003,2005.0,36511,1,3,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,0.42,0.34,0.25,0.29
4,"Baldwin County, Alabama",10.2,42119.0,76.99,23.01,01003,2005.0,36526,1,3,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,15.30,12.03,1.91,2.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145606,NaN,NaN,NaN,NaN,NaN,02198,NaN,99925,2,198,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,14.73,13.37,0.91,1.42
145607,NaN,NaN,NaN,NaN,NaN,02198,NaN,99926,2,198,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,26.26,18.18,2.80,3.39
145608,NaN,NaN,NaN,NaN,NaN,02198,NaN,99927,2,198,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,1.69,3.38,3.05,5.80
145609,NaN,NaN,NaN,NaN,NaN,02275,NaN,99903,2,275,...,8.966687e+09,6.582411e+09,100.00,100.0,100.00,100.00,1.31,4.83,5.09,6.10


In [ ]:
merged_df['Name'] = 'ZCTA5 '+ merged_df['ZCTA5']
merged_df = merged_df.rename(columns={'ZCTA5': 'ZIPS'})
main = merged_df[['Name','Poverty','Median Household Income','pct_owned','pct_rent','ZIPS','Year']]
main = main.dropna()
main['Year'] = main['Year'].astype(int)

In [106]:
df11 = df11.rename(columns={'ZIP': 'ZIPS'})
df11 = df11[['Name','Poverty','Median Household Income','pct_owned','pct_rent','ZIPS','Year']]

In [107]:
final_df = pd.concat([main,df11])
final_df

,Name,Poverty,Median Household Income,pct_owned,pct_rent,ZIPS,Year
0,ZCTA5 36480,10.2,42119.0,76.99,23.01,36480,2005
1,ZCTA5 36502,10.2,42119.0,76.99,23.01,36502,2005
2,ZCTA5 36507,10.2,42119.0,76.99,23.01,36507,2005
3,ZCTA5 36511,10.2,42119.0,76.99,23.01,36511,2005
4,ZCTA5 36526,10.2,42119.0,76.99,23.01,36526,2005
...,...,...,...,...,...,...,...
33769,ZCTA5 99923,-666666666.0,-666666666.0,100.00,0.0,99923,2021
33770,ZCTA5 99925,13.4,70625.0,73.30,26.7,99925,2021
33771,ZCTA5 99926,13.2,58229.0,80.50,19.5,99926,2021
33772,ZCTA5 99927,-666666666.0,-666666666.0,100.00,0.0,99927,2021


In [108]:
household_poverty = final_df[['Year','ZIPS','Name','Poverty','Median Household Income']]
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
household_poverty.to_csv(path + 'household_poverty_zcta5_acs.csv',index=False)

In [109]:
owned_rent = final_df[['Year','ZIPS','Name','pct_owned','pct_rent']]
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
owned_rent.to_csv(path+'owned_rent_zcta5_acs.csv', index=False)

### 3. Unemployment

##### 2011-2021

In [111]:
years1 = range(2011,2020)
variables1 = 'DP03_0009PE'
column_names1 = ['Name','pct_unemployed','State','ZIP']

years2 = [2020,2021]
column_names2 = ['Name','pct_unemployed','ZIP']

df1 = generate_df(years1, variables1, column_names1)
df2 = generate_df(years2, variables1, column_names2)

Output hidden; open in https://colab.research.google.com to view.

In [126]:
un = pd.concat([df1,df2])

##### 2005-2010

In [119]:
def generate_df_county(years, column_names):
  merged_df = pd.DataFrame()

  for year in years:
      if year == 2005 or year == 2006:
          variables = 'DP03_0009E'
      elif year == 2007 or year == 2008 or year == 2009:
          variables = 'DP03_0005PE'
      else:
          variables = 'DP03_0009PE'

      endpoint = f'https://api.census.gov/data/{year}/acs/acs1/profile?get=NAME,{variables}&for=county:*&key={api_key}'

      response = requests.get(endpoint)
      # Check if the request was successful
      if response.status_code == 200:
          data = response.json()  # Parse the response as JSON
          print(year)
          print(data)  # Display the retrieved data
      else:
          print(year)
          print(f'Request failed with status code: {response.status_code}')

      # Rename the columns and add the year
      df = pd.DataFrame(data[1:], columns=data[0])
      df = df.rename(columns=dict(zip(df.columns, column_names)))
      df['Year'] = year
      df['State']  = df['State'].astype(str).str.zfill(2)
      df['County'] = df['County'].astype(str).str.zfill(3)
      df['FIPS'] = df['State'] + df['County']

      # Convert the data to numeric
      columns_to_convert = column_names[1:-2]
      for column in columns_to_convert:
          df[column] = pd.to_numeric(df[column], errors='coerce')

      merged_df = pd.concat([merged_df,df],axis=0)

      time.sleep(5)

  return merged_df

In [120]:
years1 = [2005,2006,2007,2008,2009,2010]
column_names1 = ['Name','pct_unemployed','State','County']
df = generate_df_county(years1, column_names1)

2005
[['NAME', 'DP03_0009E', 'state', 'county'], ['Baldwin County, Alabama', '5.2', '01', '003'], ['Calhoun County, Alabama', '7.5', '01', '015'], ['Cullman County, Alabama', '6.6', '01', '043'], ['DeKalb County, Alabama', '-999999999.0', '01', '049'], ['Elmore County, Alabama', '-999999999.0', '01', '051'], ['Etowah County, Alabama', '9.0', '01', '055'], ['Houston County, Alabama', '5.5', '01', '069'], ['Jefferson County, Alabama', '7.8', '01', '073'], ['Lauderdale County, Alabama', '10.6', '01', '077'], ['Lee County, Alabama', '8.1', '01', '081'], ['Limestone County, Alabama', '-999999999.0', '01', '083'], ['Madison County, Alabama', '6.1', '01', '089'], ['Marshall County, Alabama', '-999999999.0', '01', '095'], ['Mobile County, Alabama', '6.9', '01', '097'], ['Montgomery County, Alabama', '8.9', '01', '101'], ['Morgan County, Alabama', '6.8', '01', '103'], ['St. Clair County, Alabama', '-999999999.0', '01', '115'], ['Shelby County, Alabama', '3.8', '01', '117'], ['Talladega County, 

In [121]:
merged_df = df.merge(crosswalk, on='FIPS', how='outer')
merged_df

,Name,pct_unemployed,State,County,Year,FIPS,ZCTA5,STATE,COUNTY,GEOID,...,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,"Baldwin County, Alabama",5.2,01,003,2005.0,01003,36480,1,3,1003,...,5.250712e+09,4.117522e+09,1.02,0.7,0.64,0.65,0.01,0.01,0.05,0.07
1,"Baldwin County, Alabama",5.2,01,003,2005.0,01003,36502,1,3,1003,...,5.250712e+09,4.117522e+09,1.20,1.4,11.00,11.05,0.12,0.09,1.79,2.28
2,"Baldwin County, Alabama",5.2,01,003,2005.0,01003,36507,1,3,1003,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,10.47,7.58,16.67,21.06
3,"Baldwin County, Alabama",5.2,01,003,2005.0,01003,36511,1,3,1003,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,0.42,0.34,0.25,0.29
4,"Baldwin County, Alabama",5.2,01,003,2005.0,01003,36526,1,3,1003,...,5.250712e+09,4.117522e+09,100.00,100.0,100.00,100.00,15.30,12.03,1.91,2.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145606,NaN,NaN,NaN,NaN,NaN,02198,99925,2,198,2198,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,14.73,13.37,0.91,1.42
145607,NaN,NaN,NaN,NaN,NaN,02198,99926,2,198,2198,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,26.26,18.18,2.80,3.39
145608,NaN,NaN,NaN,NaN,NaN,02198,99927,2,198,2198,...,1.989961e+10,1.016019e+10,100.00,100.0,100.00,100.00,1.69,3.38,3.05,5.80
145609,NaN,NaN,NaN,NaN,NaN,02275,99903,2,275,2275,...,8.966687e+09,6.582411e+09,100.00,100.0,100.00,100.00,1.31,4.83,5.09,6.10


In [123]:
# merged_df['Name'] = 'ZCTA5 '+ merged_df['ZCTA5']
# merged_df = merged_df.rename(columns={'ZCTA5': 'ZIPS'})
main = merged_df[['Name','pct_unemployed','ZIPS','Year']]
main = main.dropna()
main['Year'] = main['Year'].astype(int)

In [129]:
main

,Name,pct_unemployed,ZIPS,Year
0,ZCTA5 36480,5.2,36480,2005
1,ZCTA5 36502,5.2,36502,2005
2,ZCTA5 36507,5.2,36507,2005
3,ZCTA5 36511,5.2,36511,2005
4,ZCTA5 36526,5.2,36526,2005
...,...,...,...,...
121741,ZCTA5 23690,5.8,23690,2010
121742,ZCTA5 23691,5.8,23691,2010
121743,ZCTA5 23692,5.8,23692,2010
121744,ZCTA5 23693,5.8,23693,2010


In [128]:
un = un.rename(columns={'ZIP': 'ZIPS'})
un = un[['Name','pct_unemployed','ZIPS','Year']]

In [130]:
unemployed = pd.concat([main,un])
unemployed

,Name,pct_unemployed,ZIPS,Year
0,ZCTA5 36480,5.2,36480,2005
1,ZCTA5 36502,5.2,36502,2005
2,ZCTA5 36507,5.2,36507,2005
3,ZCTA5 36511,5.2,36511,2005
4,ZCTA5 36526,5.2,36526,2005
...,...,...,...,...
33769,ZCTA5 99923,0.0,99923,2021
33770,ZCTA5 99925,10.3,99925,2021
33771,ZCTA5 99926,15.9,99926,2021
33772,ZCTA5 99927,-666666666.0,99927,2021


In [132]:
unemployed = unemployed[['Year','ZIPS','Name','pct_unemployed']]
path = '/content/drive/MyDrive/EY 2022-2023 Junior/Summer 2023/INCITE Buffett Letters project/csvs/'
unemployed.to_csv(path+'unemployment_zcta5_acs.csv', index=False)